In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import matplotlib.pyplot as plt
import os

# SETUP PATH - SESUAIKAN DENGAN LOKASI DI KOMPUTERMU
base_path = "dataset"
train_dir = os.path.join(base_path, "train")
val_dir = os.path.join(base_path, "test")  # gunakan 'test' sebagai validation

# IMAGE DATA GENERATORS
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)
val_gen = ImageDataGenerator(rescale=1./255)

# LOAD DATA
train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

val_data = val_gen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# BUILD MODEL
base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# FREEZE BASE MODEL
for layer in base_model.layers:
    layer.trainable = False

# COMPILE
model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

# TRAIN
history = model.fit(
    train_data,
    epochs=15,
    validation_data=val_data
)

# SAVE TO H5
model.save("model_buah_sayur.h5")
print("✅ Model saved as 'model_buah_sayur.h5'")

# SAVE TO TFLITE
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("model_buah_sayur.tflite", "wb") as f:
    f.write(tflite_model)
print("✅ Model saved as 'model_buah_sayur.tflite'")

# SAVE LABELS
label_dict = train_data.class_indices
label_dict = {v: k for k, v in label_dict.items()}
with open("label_map.txt", "w") as f:
    for idx in sorted(label_dict):
        f.write(f"{label_dict[idx]}\n")
print("✅ Label map saved as 'label_map.txt'")

# OPTIONAL: PLOT HISTORY
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label="Train Accuracy")
plt.plot(history.history['val_accuracy'], label="Val Accuracy")
plt.title("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label="Train Loss")
plt.plot(history.history['val_loss'], label="Val Loss")
plt.title("Loss")
plt.legend()
plt.show()


Found 43324 images belonging to 77 classes.
Found 3576 images belonging to 77 classes.


C:\Users\hsant\anaconda3\envs\Kuliah-AI\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15


In [ ]:
# Visualisasi akurasi
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()
